In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = '/content/drive/MyDrive/DreamDB/Features'

In [ ]:
user_files = []

files = glob.glob(url + '/' + '*.csv')
for f in files:
    user_files.append(f)

print('No. of files: ' + str(len(user_files)))

In [ ]:
li_files = []

for f in user_files:
  df_temp = pd.read_csv(f)
  li_files.append(df_temp)

df_fe = pd.concat(li_files, axis = 0, ignore_index = True)

In [ ]:
df_fe.info()

In [ ]:
df_fe.shape

In [ ]:
df_fe.head()

In [ ]:
df_fe = df_fe.drop(columns = 'level')

In [ ]:
filters = [(df_fe.ados_preTest_total >= 0) & (df_fe.ados_preTest_total <= 6) & df_fe.ados_preTest_module == 1,
           (df_fe.ados_preTest_total >= 7) & (df_fe.ados_preTest_total <= 11) & df_fe.ados_preTest_module == 1,
           (df_fe.ados_preTest_total >= 12) & (df_fe.ados_preTest_total <= 24) & df_fe.ados_preTest_module == 1,
           (df_fe.ados_preTest_total >= 0) & (df_fe.ados_preTest_total <= 7) & df_fe.ados_preTest_module == 2,
           (df_fe.ados_preTest_total >= 8) & (df_fe.ados_preTest_total <= 11) & df_fe.ados_preTest_module == 2,
           (df_fe.ados_preTest_total >= 12) & (df_fe.ados_preTest_total <= 24) & df_fe.ados_preTest_module == 2,
                ]
values = [0, 1, 2, 0, 1, 2]
df_fe['level'] = np.select(filters, values)

In [ ]:
df_fe['level'].value_counts()

In [ ]:
data = [122736, 474180]
labels = ['ASD (AS, PDD)', 'Autistic Disorder']
fig1, ax1 = plt.subplots(figsize=(6, 6))
ax1.pie(data, labels = labels, explode = (0.1, 0.1), autopct='%1.1f%%', shadow=True, startangle=90, colors=sns.color_palette('Blues_d'), textprops={'fontsize': 16})
ax1.axis('equal')
plt.show()

In [ ]:
df_fe['age_group'].value_counts()

In [ ]:
data = [28662, 156045, 195004, 156999, 60206]
labels = ['1', '2', '3', '4', '5']
fig1, ax1 = plt.subplots(figsize=(6, 6))
ax1.pie(data, labels = labels, explode = (0.1, 0.1, 0.1, 0.1, 0.1), autopct='%1.1f%%', shadow=True, startangle=90, colors=sns.color_palette('Blues_d'), textprops={'fontsize': 16})
ax1.axis('equal')
plt.show()

In [ ]:
df_fe['ados_preTest_module'].value_counts()

In [ ]:
df_fe['condition'].value_counts()

In [ ]:
data = [122736, 474180]
labels = ['ASD (AS, PDD)', 'Autistic Disorder']
fig1, ax1 = plt.subplots()
ax1.pie(data, labels = labels, explode = (0.1, 0.1), autopct='%1.1f%%', shadow=True, startangle=90, colors=sns.color_palette('Blues_d'))
ax1.axis('equal')
plt.show()

In [ ]:
a = df_fe['task_ability'].value_counts()

In [ ]:
print(a.head())

In [ ]:
df_fe.shape

In [ ]:
df_temp = df_fe.copy()

In [ ]:
df_temp = df_temp.dropna()

In [ ]:
df_temp.shape

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

In [ ]:
col_drop_list = ['participant_gender', 'ados_preTest_communication', 'ados_preTest_interaction', 'ados_preTest_module', 'ados_preTest_play', 'ados_preTest_protocol',
            'ados_preTest_socialCommunicationQuestionnaire', 'ados_preTest_stereotype', 'condition', 'ados_preTest_total']

In [ ]:
df_temp = df_temp.drop(columns = col_drop_list)

In [ ]:
df_temp.shape

In [ ]:
X_train = df_temp.iloc[:,1:-1]

In [ ]:
X_train.head()

In [ ]:
df_temp.head()

In [ ]:
Y_train = df_temp.iloc[:,-1]

In [ ]:
Y_train.head()

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
skfcv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 11)

In [ ]:
RF_fs = RandomForestClassifier(random_state=11, verbose=2, class_weight='balanced_subsample',n_estimators=50)

In [ ]:
from sklearn.svm import SVC
SVM = SVC(random_state=11, verbose=2,)

In [ ]:
rfecv=RFECV(estimator=RF_fs, min_features_to_select=53, cv=skfcv, verbose=2, step=10,scoring='accuracy')

In [ ]:
score=['accuracy','precision_macro','recall_macro','f1_macro']

In [ ]:
fs = rfecv.fit(X_train,Y_train)

In [ ]:
fs.ranking_

In [ ]:
fs.support_

In [ ]:
fs.n_features_

In [ ]:
fs.grid_scores_

In [ ]:
fs.cv_results_

In [ ]:
feature_names = X_train.columns

In [ ]:
selected_features = feature_names[fs.support_].tolist()

In [ ]:
print(selected_features)

In [ ]:
X_fs=fs.transform(X_train)

In [ ]:
print(X_fs)

In [ ]:
X_fs=pd.DataFrame(X_fs,columns=selected_features)

In [ ]:
fs_df=pd.DataFrame(selected_features,columns=["Feature label"])

In [ ]:
fs_df["Feature Importance"]=fs.estimator_.feature_importances_

In [ ]:
fs_df=fs_df.sort_values(by='Feature Importance',ascending=False)

In [ ]:
f,ax=plt.subplots(figsize=(20,10))
plt.xticks(rotation='vertical')
sns.barplot(x="Feature label",y="Feature Importance",data=fs_df,palette="Blues_d")
ax.set_frame_on(False)

In [ ]:
X_fs.to_csv('/content/drive/MyDrive/DreamDB/fs.csv',columns=X_fs.columns,index=False)

In [ ]:
RF=RandomForestClassifier(criterion='entropy', max_features='log2', random_state=11, verbose=2, class_weight='balanced_subsample',n_estimators=100,max_depth=500)

In [ ]:
scores=cross_validate(SVM,X_fs,Y_train,scoring=score,cv=skfcv)

In [ ]:
print(scores)

In [ ]:
import sklearn
print(sklearn.metrics.SCORERS.keys())

In [ ]:
X_fs=pd.read_csv('/content/drive/MyDrive/DreamDB/fs.csv')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor

score1=['neg_mean_squared_error','neg_root_mean_squared_error','neg_mean_absolute_error','r2','neg_mean_absolute_percentage_error']

In [ ]:
RFR=RandomForestRegressor(max_features='log2', random_state=11, verbose=2, n_estimators=200,max_depth=500)

In [ ]:
scores1=cross_validate(RFR,X_fs,Y_train,scoring=score1,cv=skfcv)

In [ ]:
print(scores1)